## Load libraries

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import glob
# plt.rcParams['figure.figsize'] = (25, 6)
%matplotlib inline

## Process datasets

### Merge the dataset 20171129
1. Merge the dataset to point_spread.csv and matches.csv
2. Unify the team name
  * JAX -> JAC (NFL use JAX, but just for unification)
  * Buffalo -> BUF
  * LAC -> SD
  * LA -> STL

### Read feature datasets 
from 2009 to 2017

In [2]:
play_cols = ['GameID','Drive','qtr','down','yrdline100','posteam','DefensiveTeam','TimeSecs','sp','ScoreDiff',
            'HomeTeam','AwayTeam']

In [3]:
spread_cols = ['eid','season','week','Home','Away','HomeScore','AwayScore','Day','Time','Favorite','Underdog','Spread','CoverOrNot']

In [4]:
data_files = glob.glob('data/new_data/pbp_20*.csv') # from 2009 to 2017

pbp_data = pd.DataFrame()
for f in data_files:
    d = pd.read_csv(f, usecols=play_cols)
    pbp_data = pd.concat([pbp_data, d])
pbp_data.shape

(388476, 12)

### Merge point spread data

In [5]:
spread_data = pd.read_csv('data/point_spread.csv')

In [6]:
spread_data.head()

,GameID,season,week,Home,Away,HomeScore,AwayScore,Day,Time,Favorite,Underdog,Spread,CoverOrNot
0,2009091000,2009,1,PIT,TEN,13.0,10.0,Thu,8:30,PIT,TEN,-6.5,-1.0
1,2009091304,2009,1,CLE,MIN,20.0,34.0,Sun,1:00,MIN,CLE,-4.0,1.0
2,2009091307,2009,1,NO,DET,45.0,27.0,Sun,1:00,NO,DET,-14.0,1.0
3,2009091308,2009,1,TB,DAL,21.0,34.0,Sun,1:00,DAL,TB,-5.5,1.0
4,2009091305,2009,1,HOU,NYJ,7.0,24.0,Sun,1:00,HOU,NYJ,-4.5,-1.0


In [7]:
data = pd.merge(pbp_data, spread_data, on='GameID', how='left')

### Deal with missing data

In [8]:
data = data.dropna(axis=0, how='any')

In [9]:
data.shape

(319576, 24)

In [10]:
data = data[data['TimeSecs'] >= 0]

In [11]:
data.head()

,GameID,Drive,qtr,down,TimeSecs,yrdline100,posteam,DefensiveTeam,sp,ScoreDiff,...,Home,Away,HomeScore,AwayScore,Day,Time,Favorite,Underdog,Spread,CoverOrNot
1,2009091000,1,1,1.0,3593.0,58.0,PIT,TEN,0,0.0,...,PIT,TEN,13.0,10.0,Thu,8:30,PIT,TEN,-6.5,-1.0
2,2009091000,1,1,2.0,3556.0,53.0,PIT,TEN,0,0.0,...,PIT,TEN,13.0,10.0,Thu,8:30,PIT,TEN,-6.5,-1.0
3,2009091000,1,1,3.0,3515.0,56.0,PIT,TEN,0,0.0,...,PIT,TEN,13.0,10.0,Thu,8:30,PIT,TEN,-6.5,-1.0
4,2009091000,1,1,4.0,3507.0,56.0,PIT,TEN,0,0.0,...,PIT,TEN,13.0,10.0,Thu,8:30,PIT,TEN,-6.5,-1.0
5,2009091000,2,1,1.0,3496.0,98.0,TEN,PIT,0,0.0,...,PIT,TEN,13.0,10.0,Thu,8:30,PIT,TEN,-6.5,-1.0


#### Columns to include in the formation of buckets [Down, TimeSecs, YdsToGo and Score Diff]

In [12]:
data['homeScoreDiff'] = data[['posteam','ScoreDiff','HomeTeam']].apply(
    lambda x: x['ScoreDiff'] if x['posteam']==x['HomeTeam'] else (-x['ScoreDiff']),axis=1)

In [13]:
data['favTeamOnBall'] = data[['Favorite','posteam']].apply(lambda x: 1 if x['Favorite']==x['posteam'] else -1, axis=1)

In [14]:
data.head()

,GameID,Drive,qtr,down,TimeSecs,yrdline100,posteam,DefensiveTeam,sp,ScoreDiff,...,HomeScore,AwayScore,Day,Time,Favorite,Underdog,Spread,CoverOrNot,homeScoreDiff,favTeamOnBall
1,2009091000,1,1,1.0,3593.0,58.0,PIT,TEN,0,0.0,...,13.0,10.0,Thu,8:30,PIT,TEN,-6.5,-1.0,0.0,1
2,2009091000,1,1,2.0,3556.0,53.0,PIT,TEN,0,0.0,...,13.0,10.0,Thu,8:30,PIT,TEN,-6.5,-1.0,0.0,1
3,2009091000,1,1,3.0,3515.0,56.0,PIT,TEN,0,0.0,...,13.0,10.0,Thu,8:30,PIT,TEN,-6.5,-1.0,0.0,1
4,2009091000,1,1,4.0,3507.0,56.0,PIT,TEN,0,0.0,...,13.0,10.0,Thu,8:30,PIT,TEN,-6.5,-1.0,0.0,1
5,2009091000,2,1,1.0,3496.0,98.0,TEN,PIT,0,0.0,...,13.0,10.0,Thu,8:30,PIT,TEN,-6.5,-1.0,-0.0,-1


The range of each variable
- down = [1,4] Four buckets from here
- ydstogo = [0,100] 10 buckets from here
- homeScoreDiff = [-60, 60] 30 buckets from here 
- TimeSecs = [0,3600] 180 buckets from here

Total buckets = 180\*30\*10\*4 = 216000

### Notes 
1. Count a minute a bucket: [0,3600] -> 60 buckets {Done}
2. Add a flag bucket for who possessing the ball, favorite or underdog {Not Done}


In [15]:
data['DownClass'] = pd.cut(data['down'],4, labels=False)

In [16]:
data['TimeSecsClass'] = pd.cut(data['TimeSecs'],60, labels=False)

In [17]:
data['yrdline100Class'] = pd.cut(data['yrdline100'],10, labels=False)

In [18]:
data['homeScoreDiffClass'] = pd.cut(data['homeScoreDiff'],30, labels=False)

In [35]:
class_names = ['homeScoreDiffClass','yrdline100Class','TimeSecsClass','DownClass']

In [36]:
# Count the total plays in each bucket irrespective of bet_cleared or not
temp = data.groupby(class_names).size().reset_index(name='counts')

Now temp contains the count of each existing class. From here, we've to add 0 for values of the classes that don't exist. The following code does that.

In [40]:
# Counting the values in each bucket
bucket_full = pd.pivot_table(temp,
                           index=class_names,
                           values='counts',
                           fill_value = 0,
                           dropna=False,
                           aggfunc=np.sum)

new_table stores the count for each class.

In [42]:
bucket_full['counts'] = bucket_full['counts']+2

In [43]:
bucket_full.head()

counts
homeScoreDiffClass yrdline100Class TimeSecsClass DownClass        
0                  0               0             0               2
                                                 1               2
                                                 2               2
                                                 3               2
                                   1             0               2

In [44]:
full_buc_count = list(bucket_full['counts'])

In [45]:
len(full_buc_count)

72000

Now we will work to get the count for each bucket with only the matches for which the bet was cleared

In [46]:
class_names = ['CoverOrNot','homeScoreDiffClass','yrdline100Class','TimeSecsClass','DownClass']

In [47]:
temp = data.groupby(class_names).size().reset_index(name='counts')

In [48]:
temp = temp[temp['CoverOrNot']==1]

In [49]:
# Counting the plays in each bucket for which the bet was cleared
bucket_half = pd.pivot_table(temp,
                           index=class_names,
                           values='counts',
                           fill_value = 0,
                           dropna=False,
                           aggfunc=np.sum)

In [50]:
bucket_half['counts'] = bucket_half['counts']+1

In [51]:
half_buc_count = list(bucket_half['counts'])

In [52]:
len(full_buc_count)

72000

In [53]:
len(half_buc_count)

72000

In [57]:
# Final Probability vector for each bucket
prob_buc = [float(half_buc_count[i])/float(full_buc_count[i]) for i in range(len(full_buc_count))]

**prob_buc** stores the probability of bet clearing probability of each bucket

In [31]:
data.iloc[100]

GameID                2009091000
Drive                         18
qtr                            3
down                           1
TimeSecs                    1208
yrdline100                    90
posteam                      TEN
DefensiveTeam                PIT
sp                             0
ScoreDiff                      0
HomeTeam                     PIT
AwayTeam                     TEN
season                      2009
week                           1
Home                         PIT
Away                         TEN
HomeScore                     13
AwayScore                     10
Day                          Thu
Time                        8:30
Favorite                     PIT
Underdog                     TEN
Spread                      -6.5
CoverOrNot                    -1
homeScoreDiff                 -0
DownClass                      0
TimeSecsClass                 20
yrdline100Class                9
homeScoreDiffClass            12
Name: 112, dtype: object

### Plot the relation along different features

In [48]:
# data_p = data[data["GameID"]]
avai_idx = [int(str(eid)[:4]) > 2012 for eid in data["GameID"]]
data.iloc[avai_idx]

,GameID,Drive,qtr,down,TimeSecs,yrdline100,posteam,DefensiveTeam,sp,ScoreDiff,HomeTeam,AwayTeam,homeScoreDiff,DownClass,TimeSecsClass,yrdline100Class,homeScoreDiffClass
1,2013090500,1,1,1.0,3600.0,80.0,BAL,DEN,0,0.0,DEN,BAL,-0.0,0,179,8,12
2,2013090500,1,1,2.0,3554.0,81.0,BAL,DEN,0,0.0,DEN,BAL,-0.0,1,178,8,12
3,2013090500,1,1,3.0,3518.0,78.0,BAL,DEN,0,0.0,DEN,BAL,-0.0,2,176,7,12
4,2013090500,1,1,4.0,3498.0,72.0,BAL,DEN,0,0.0,DEN,BAL,-0.0,3,175,7,12
5,2013090500,2,1,1.0,3490.0,77.0,DEN,BAL,0,0.0,DEN,BAL,0.0,0,175,7,12
6,2013090500,2,1,2.0,3485.0,77.0,DEN,BAL,0,0.0,DEN,BAL,0.0,1,175,7,12
7,2013090500,2,1,3.0,3445.0,76.0,DEN,BAL,0,0.0,DEN,BAL,0.0,2,173,7,12
8,2013090500,2,1,1.0,3424.0,66.0,DEN,BAL,0,0.0,DEN,BAL,0.0,0,172,6,12
9,2013090500,2,1,2.0,3418.0,66.0,DEN,BAL,0,0.0,DEN,BAL,0.0,1,172,6,12
10,2013090500,2,1,3.0,3384.0,59.0,DEN,BAL,0,0.0,DEN,BAL,0.0,2,171,5,12
